In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from torchsummary import summary

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
img = torch.randn(1,3,256,256).to(device)

In [4]:
class genConv2D(nn.Module):
    def __init__(self):
        super(genConv2D, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=32,kernel_size=5,stride = 1,padding = 0, bias = False)
        
    def forward(self,x):
        return F.relu(self.conv1(x))
    
genConv = genConv2D().to(device)

In [5]:
class depthConv2D(nn.Module):
    def __init__(self):
        super(depthConv2D, self).__init__()
        
        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, inp, 5, stride, 0, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),
    
                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True),
            )
        
        self.conv1 = conv_dw(3, 32, 1)
    
    def forward(self,x):
        return self.conv1(x)
depthConv = depthConv2D().to(device)

In [6]:
def print_num_parameters(model):
    params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Total number of parameters is: {}".format(params))

In [7]:
print_num_parameters(genConv)

Total number of parameters is: 2400


In [8]:
print_num_parameters(depthConv)

Total number of parameters is: 241


In [9]:
summary(depthConv, (3,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 3, 124, 124]              75
       BatchNorm2d-2          [-1, 3, 124, 124]               6
              ReLU-3          [-1, 3, 124, 124]               0
            Conv2d-4         [-1, 32, 124, 124]              96
       BatchNorm2d-5         [-1, 32, 124, 124]              64
              ReLU-6         [-1, 32, 124, 124]               0
Total params: 241
Trainable params: 241
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 12.32
Params size (MB): 0.00
Estimated Total Size (MB): 12.51
----------------------------------------------------------------


In [10]:
summary(genConv, (3,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 124, 124]           2,400
Total params: 2,400
Trainable params: 2,400
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 3.75
Params size (MB): 0.01
Estimated Total Size (MB): 3.95
----------------------------------------------------------------


In [11]:
genConv(img).size()

torch.Size([1, 32, 252, 252])

In [12]:
depthConv(img).size()

torch.Size([1, 32, 252, 252])